Url: https://tbrain.trendmicro.com.tw/Competitions/Details/2

In [22]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit,trim
from pyspark.sql.window import Window

In [23]:
global Path
global DoEval #是否進行模型評估

DoEval = True
next_date_range = ["20180514", "20180515", "20180516", "20180517", "20180518"] #設定預測區間
ignore_dates = [""]#設定排除日(如端午節)
#next_date_range = ["20180507", "20180508", "20180509", "20180510", "20180511"] #設定預測區間
#ignore_dates = [""]#設定排除日(如端午節)
#next_date_range = ["20180430", "20180501", "20180502", "20180503", "20180504"] #設定預測區間
#ignore_dates = ["20180501"]#設定排除日(如端午節)
predict_start_date = next_date_range[0]

if sc.master[0:5]=="local":
    #Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180511"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180518"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"
else:
    Path = "hdfs://master:9000/user/hduser"
Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180518"

In [24]:
sc.master

'spark://sparklab:7077'

In [25]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}

In [26]:
#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [27]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [28]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19306
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [29]:
#EDA
#range of date
tetfp_dt.describe('etf_date').show()

+-------+--------------------+
|summary|            etf_date|
+-------+--------------------+
|  count|               19306|
|   mean|2.0153649878068995E7|
| stddev|    15917.8419890693|
|    min|            20130102|
|    max|            20180518|
+-------+--------------------+



In [30]:
# print("starting import taetfp.csv(台灣18檔ETF調整後股價資料)...")
# taetfp_dt=read_data("taetfp.csv", col_alias_etf)

In [31]:
# print("starting import tsharep.csv(台灣個股股價資料)...")
# tsharep_dt=read_data("tsharep.csv", col_alias_stock)

In [32]:
# print("starting import tasharep.csv(台灣個股調整後股價資料)...")
# tasharep_dt=read_data("tasharep.csv", col_alias_stock)

In [33]:
import sys
from pyspark.sql.functions import lag, col, avg,collect_list, lit
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
wsSpec_etf_close_price_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, -1)
wsSpec_etf_dif_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, 0)
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)
#計算EMA的udf
def calculate_ema_native(close_p_list, window_len):
    #透過歷史收盤價計算
    if len(close_p_list) < window_len:
        return None
    elif len(close_p_list) == window_len:
        #if len of list = win_len then return avg, 
        return avg_list(close_p_list)
    else:
        #else EMA[t] =(EMA[t-1]*(win_len-1)+close[t]*2)/(win_len+1)
        ema = avg_list(close_p_list[:window_len])
        for price in close_p_list[window_len:]:
            ema = (ema*(window_len-1)+price*2)/(window_len+1)
        return ema
calculate_ema=udf(calculate_ema_native, DoubleType())
#計算BIAS的udf
def calculate_bias(close_p_list):
    #計算前日收盤價與N日均線之差比: (close price - MA)/MA   ,Paper 建議用20日MA
    #因要預測今日的收盤價，故計算前日收盤價與前20日均線
    if len(close_p_list) < 21:
        return None
    else:
        list_len = len(close_p_list)
        p_close = close_p_list[-1]
        cal_list = close_p_list[list_len-21: list_len-1]
        return p_close - avg_list(cal_list)
calculate_bias=udf(calculate_bias, DoubleType())

def get_min_max_last(p_list):
    #找出list中最大最小和最後一個值, 回傳(min, max, last)
    return (min(p_list), max(p_list), p_list[-1])
def calculate_raw_rsv(p_list):
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    p_min, p_max, p_last = get_min_max_last(p_list)
    rsv = (p_last - p_min)/(p_max - p_min)
    return rsv
def calculate_rsv(p_9_list, k_prev, d_prev):
    #計算加權後的RSV，p_9_list=>9日收盤價
    rrsv = calculate_raw_rsv(p_9_list)
    k_curr = (1/3)*rrsv + (2/3)*k_prev
    d_curr = (1/3)*k_curr + (2/3)*d_prev
    return [k_curr, d_curr]
#計算隨機指標（Stochastic Oscillator，KD），原名%K&%D
def calculate_KD(close_p_list):
    win_len = 9 #看過去 9 日值
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    #K_curr = 1/3*RSV + 2/3*K_prev
    #D_curr = 1/3*K_curr + 2/3*D_prev
    if len(close_p_list) < win_len:
        return None
    elif len(close_p_list) == win_len:
        #無前日K, D時，以0.5帶入
        return calculate_rsv(close_p_list, 0.5, 0.5)
    else:
        kds = calculate_rsv(close_p_list[0:9], 0.5, 0.5)
        for idx in range(1, (len(close_p_list)+1-9)):
            p_9_list = close_p_list[idx: idx+9]
            kds = calculate_rsv(p_9_list, kds[0], kds[1])
        return kds
calculate_KD=udf(calculate_KD, ArrayType(DoubleType()))

#計算差離值DIF = 12日EMA - 26日EMA
def calculate_DIF_native(close_p_list):
    if len(close_p_list) < 26:
        return None
    else:
        ema12 = calculate_ema_native(close_p_list, 12)
        ema26 = calculate_ema_native(close_p_list, 26)
        return ema12 - ema26
calculate_DIF=udf(calculate_DIF_native, DoubleType())

#計算MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
def calculate_MACD(dif_list, dif_curr):
    win_len = 9
    if len(dif_list) < win_len:
        return None
    elif len(dif_list) == win_len:
        #if len of list = win_len then return avg, 
        return avg_list(dif_list)
    else:
        #MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
        macd = avg_list(dif_list[:win_len])
        for price in dif_list[win_len:]:
            macd = (macd*(win_len-1)+dif_curr*2)/(win_len+1)
        return macd
calculate_MACD=udf(calculate_MACD, DoubleType())

#計算相對強弱指數(RSI)
def calculate_RSI(close_p_list):
    win_len = 9
    if len(close_p_list) < (win_len + 1):
        return None
    else:
        cur_list = close_p_list[1:]
        prv_list = close_p_list[0:-1]
        p_dif_list = list(map(lambda x,y : x - y, cur_list, prv_list)) #dif list
        u_list = []
        d_list = []
        for dif in p_dif_list:
            if dif == 0:
                #若兩天價格相同，則U及D皆等於零
                u_list.append(0)
                d_list.append(0)
            elif dif > 0:
                #在價格上升的日子, U = diff, D = 0
                u_list.append(dif)
                d_list.append(0)
            else:
                #在價格下跌的日子, U = 0, D = abs(diff)
                u_list.append(0)
                d_list.append(abs(dif))
        #RSI = ema(u,9)/(ema(u,9)+ema(d,9))
        ema_u = calculate_ema_native(u_list, win_len)
        ema_d = calculate_ema_native(d_list, win_len)
        return ema_u/(ema_u + ema_d)
calculate_RSI=udf(calculate_RSI, DoubleType())

#計算威廉指標（Williams %R）
def calculate_WR(close_p_list):
    win_len = 9
    if len(close_p_list) < win_len:
        return None
    else:
        p_list = close_p_list[len(close_p_list) - win_len :]
        return 1.0 - calculate_raw_rsv(p_list)
calculate_WR=udf(calculate_WR, DoubleType())

#calculate profit of n-days
def calculate_days_profit_natvie(close_p_list, days):
    if len(close_p_list) < days+1:
        return 0.0
    cal_list = close_p_list[len(close_p_list)-days-1 : ]
    #formula: [(1+daily_profit)*]-1
    res = 1.0
    for idx in range(0, days):
        res = res * (1 + (cal_list[idx+1]/cal_list[idx]-1.0))
    res = res -1.0
    return res
calculate_days_profit=udf(calculate_days_profit_natvie, DoubleType())

from datetime import datetime
def get_weekday_native(date_str):
    return datetime.strptime(date_str.strip(), "%Y%m%d").weekday()+0.0
get_weekday = udf(get_weekday_native , DoubleType())

In [34]:
#calculate ema [5,10,20] #cannot remove row_idx, row_idx for next window usage
tetfp_dt2=tetfp_dt.withColumn("row_idx", rank().over(wsSpec_etf)) \
    .withColumn("close_price_raw", collect_list(col('etf_close')).over(wsSpec_etf_close_price_raw)) \
    .withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("EMA30", calculate_ema(col("close_price_raw"), lit(30))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("DIF", calculate_DIF(col("close_price_raw"))) \
    .withColumn("dif_list", collect_list(col('DIF')).over(wsSpec_etf_dif_raw)) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("P_1", calculate_days_profit(col("close_price_raw"), lit(1))) \
    .withColumn("P_5", calculate_days_profit(col("close_price_raw"), lit(5))) \
    .withColumn("weekday", get_weekday(col("etf_date"))) \
    .withColumn("WR", calculate_WR(col("close_price_raw")))

tetfp_dt2.cache()
tetfp_dt2.printSchema()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- close_price_raw: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- EMA30: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- KD: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- dif_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- P_1: double (n

In [35]:
#tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_close", "MACD", "RSI", "WR", 'weekday') \
#             .show()
#          .toPandas().to_csv(Path.replace("file:","") + "/taetfp_BIAS.csv",index=False)

In [36]:
#匯出成pandas
# etf_pd_50 = tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_date" ,"etf_close", "EMA5", "EMA10", "EMA20", "BIAS") \
#                 .toPandas()

In [37]:
#圖形化顯示
#aetf_pd_50.set_index('etf_date') #set_index 後配合 loc select 出區段資料來看
# etf_pd_50_part = etf_pd_50.loc[1200:]
# etf_pd_50_part.etf_close.plot(x='row_idx', y='etf_close', style='b--', label="etf_close")
# etf_pd_50_part.EMA5.plot(x='row_idx', y='EMA5', label="EMA5", style='r-')
# etf_pd_50_part.EMA10.plot(x='row_idx', y='EMA10', label="EMA10", style='g-')
# etf_pd_50_part.EMA20.plot(x='row_idx', y='EMA20', label="EMA20", style='y-')
# plt.legend()
# plt.show()

In [38]:
#計算各欄位與收盤價之相關性
# corr_cols = ['EMA5','EMA10','EMA20','BIAS','K','D']
# for col in corr_cols:
#     print('corr between ', col , ' and etf_close: ', str(tetfp_dt2.corr(col, 'etf_close')))

In [39]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
tot_dt = tetfp_dt2.filter("MACD is not null and EMA30 is not null") \
    .select("etf_id", "etf_date", "row_idx", "EMA5", "EMA10", "EMA20", "EMA30", "BIAS", "K", "D", "DIF", "MACD", "RSI", 
            "P_1", "P_5", (col("WR")*-1.0).alias("WR"), "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)

In [40]:
chaos_file_name="etf_extraFeatures_0518_L37.csv"
#$$$
chaos_raw_data = spark.read.csv(Path + "/" + chaos_file_name, header=True, sep=",") \
    .filter("etf_date > '20140101'")
print("Total " + chaos_file_name + " count: " + str(chaos_raw_data.count()))
chaos_raw_data.printSchema()
chaos_raw_data.show(10)

Total etf_extraFeatures_0518_L37.csv count: 15816
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_close: string (nullable = true)
 |-- bias80: string (nullable = true)
 |-- d80: string (nullable = true)
 |-- dif80: string (nullable = true)
 |-- k80: string (nullable = true)
 |-- macd80: string (nullable = true)
 |-- p1_80: string (nullable = true)
 |-- p5_80: string (nullable = true)
 |-- rsi80: string (nullable = true)
 |-- wr80: string (nullable = true)
 |-- close80: string (nullable = true)

+------+--------+---------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+
|etf_id|etf_date|etf_close|            bias80|                d80|              dif80|                k80|             macd80|              p1_80|             p5_80|             rsi80|               wr80|           close80|
+------+

In [41]:
chaos_dt = chaos_raw_data.filter("wr80 != ''") \
    .select("etf_id", "etf_date", col("bias80").cast("Double"), col("d80").cast("Double"), col("dif80").cast("Double"), 
           col("k80").cast("Double"), col("macd80").cast("Double"), col("p1_80").cast("Double"), 
            col("p5_80").cast("Double"), col("rsi80").cast("Double"), (col("wr80").cast("Double")*-1.0).alias("wr80"),
           col("close80").cast("Double"))
chaos_dt.printSchema()
chaos_dt.show(10)

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- bias80: double (nullable = true)
 |-- d80: double (nullable = true)
 |-- dif80: double (nullable = true)
 |-- k80: double (nullable = true)
 |-- macd80: double (nullable = true)
 |-- p1_80: double (nullable = true)
 |-- p5_80: double (nullable = true)
 |-- rsi80: double (nullable = true)
 |-- wr80: double (nullable = true)
 |-- close80: double (nullable = true)

+------+--------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+
|etf_id|etf_date|            bias80|                d80|              dif80|                k80|             macd80|              p1_80|             p5_80|             rsi80|              wr80|           close80|
+------+--------+------------------+-------------------+-------------------+-------------------+-------------------+-----

In [42]:
tot_dt2 = tot_dt.join(chaos_dt, 
                        [trim(tot_dt.etf_id) == chaos_dt.etf_id , trim(tot_dt.etf_date) == chaos_dt.etf_date], "inner") \
    .drop(chaos_dt.etf_id).drop(chaos_dt.etf_date)

tot_dt2.printSchema()
tot_dt2.show(10)

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- EMA30: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- P_1: double (nullable = true)
 |-- P_5: double (nullable = true)
 |-- WR: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- bias80: double (nullable = true)
 |-- d80: double (nullable = true)
 |-- dif80: double (nullable = true)
 |-- k80: double (nullable = true)
 |-- macd80: double (nullable = true)
 |-- p1_80: double (nullable = true)
 |-- p5_80: double (nullable = true)
 |-- rsi80: double (nullable = true)
 |-- wr80: double (nullable = true)
 |-- close80: double (nullable = true)

+------

In [43]:
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#$$$將Feature合併為Vector 並作標準化
assembler = VectorAssembler(
    inputCols=["EMA5", "EMA10", "EMA20", "EMA30", "BIAS", "K", "D", "DIF", "MACD", 
                "RSI","P_1", "P_5", "bias80", "d80", "dif80", "k80", "macd80", 
                "p1_80", "p5_80", "rsi80", "WR", "wr80"],
    outputCol="features")
tot_dt_1 = assembler.transform(tot_dt2)
#minmax_scaler = MinMaxScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = minmax_scaler.fit(tot_dt_1)
#std_scaler = StandardScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = std_scaler.fit(tot_dt_1)
#tot_dt_scale = scaler_model.transform(tot_dt_1)
tot_dt_scale=tot_dt_1.withColumn("stdFeatures", col("features")) #$$$ 測試不作標準化

In [44]:
###### 取出4/16~4/27 (共兩週資料作為測試集)
train_dt = tot_dt_scale.filter("etf_date < '" + predict_start_date + "' and MACD is not null and EMA30 is not null") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
test_dt = tot_dt_scale.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
print('train count: ', str(train_dt.count()), ', test count: ', str(test_dt.count()))
train_dt.show(10)
test_dt.show(10)

train count:  15602 , test count:  90
+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------+--------+-------+--------------------+---------+
|0050   |20140102|    247|[58.3769898121632...|    58.55|
|0050   |20140103|    248|[58.4346598747754...|    57.85|
|0050   |20140106|    249|[58.2397732498503...|     57.7|
|0050   |20140107|    250|[58.0598488332335...|     57.7|
|0050   |20140108|    251|[57.9398992221557...|     57.8|
|0050   |20140109|    252|[57.8932661481038...|    57.55|
|0050   |20140110|    253|[57.7788440987358...|    57.55|
|0050   |20140113|    254|[57.7025627324905...|     57.9|
|0050   |20140114|    255|[57.7683751549937...|     57.8|
|0050   |20140115|    256|[57.7789167699958...|     58.4|
+-------+--------+-------+--------------------+---------+
only showing top 10 rows

+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------

In [45]:
#test_dt.show(20, False)

In [46]:
train_dt.cache()
test_dt.cache()

DataFrame[etf_id: string, etf_date: string, row_idx: int, stdFeatures: vector, etf_close: double]

In [47]:
#取出etf的distinct id
etf_ids = []
for row in train_dt.select("etf_id").distinct().orderBy("etf_id").collect():
    etf_ids.append(row["etf_id"])
etf_ids

['0050   ',
 '0051   ',
 '0052   ',
 '0053   ',
 '0054   ',
 '0055   ',
 '0056   ',
 '0057   ',
 '0058   ',
 '0059   ',
 '006201 ',
 '006203 ',
 '006204 ',
 '006208 ',
 '00690  ',
 '00692  ',
 '00701  ',
 '00713  ']

In [48]:
#訓練Model及評估(RandomForestRegressor - etf_id wise) 
#-- accuracy:0.48, (RMSE) on test data = 0.938023 --> using stdScaler
#-- accuracy:0.48, (RMSE) on test data = 0.87064 --> no Scaler
#-- accuracy:0.56, (RMSE) on test data = 0.92546 --> use minmaxScaler
#-- accuracy:0.513812 , (RMSE) on test data = 0.597397 --> use minmaxScaler and no Scaler(full data)
from pyspark.ml.regression import RandomForestRegressor
#$$$default param of RF: maxDepth=5, maxBins=32, numTrees=20
rf = RandomForestRegressor(featuresCol="stdFeatures",labelCol="etf_close", 
                           maxDepth=12, maxBins=64)
#記下對應etf_id的所有model
model_map = {}
for etfid in etf_ids:
    train_data = train_dt.filter("etf_id='" + etfid + "'")
    rf_model = rf.fit(train_data)
    model_map.update({etfid: rf_model})

In [49]:
#find last records of all etf to be the base of next record
tetf_dt_prod = tetfp_dt2.filter("etf_date < '" + predict_start_date + "' and MACD is not null")
tetf_max_idx = tetf_dt_prod.groupBy("etf_id").max("row_idx")
tetf_max = tetf_max_idx.select(col("etf_id"), col("max(row_idx)").cast("Double").alias("row_idx")) \
    .join(tetf_dt_prod, ["etf_id", "row_idx"], "inner") \
    .select("etf_id", "etf_date", "row_idx", "etf_close", "close_price_raw", "dif_list") \
    .orderBy('etf_id')

tetf_max.show()

+-------+--------+-------+---------+--------------------+--------------------+
| etf_id|etf_date|row_idx|etf_close|     close_price_raw|            dif_list|
+-------+--------+-------+---------+--------------------+--------------------+
|0050   |20180511| 1312.0|     81.6|[54.4, 54.85, 54....|[0.35882671868262...|
|0051   |20180511| 1312.0|    32.79|[26.09, 26.12, 26...|[0.23810818793990...|
|0052   |20180511| 1312.0|     53.0|[32.72, 32.12, 32...|[0.30196772810150...|
|0053   |20180511| 1312.0|    35.48|[23.26, 23.11, 22...|[0.15597485321855...|
|0054   |20180511| 1312.0|    23.76|[19.4, 19.37, 19....|[0.09327655672724...|
|0055   |20180511| 1312.0|    17.36|[11.47, 11.49, 11...|[0.31374969126963...|
|0056   |20180511| 1312.0|     25.6|[22.95, 23.06, 22...|[0.31828091002715...|
|0057   |20180511| 1312.0|    50.25|[31.94, 31.93, 31...|[0.20109705259728...|
|0058   |20180511| 1312.0|    46.84|[32.12, 31.96, 32...|[-0.1346882469799...|
|0059   |20180511| 1312.0|    42.52|[24.65, 24.67, 2

In [52]:
#建立產生feature之方法
def create_feature(etf_id, date, orig_dt, chaos_dt):
    new_dt = orig_dt.withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("EMA30", calculate_ema(col("close_price_raw"), lit(30))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("P_1", calculate_days_profit(col("close_price_raw"), lit(1))) \
    .withColumn("P_5", calculate_days_profit(col("close_price_raw"), lit(5))) \
    .withColumn("weekday", get_weekday(col("etf_date"))) \
    .withColumn("WR", calculate_WR(col("close_price_raw")))
    filter_str = "etf_id='"+etf_id+"' and etf_date='"+date+"'"
    #print('filter_str: ', filter_str)
    chaos_pd = chaos_dt.filter(filter_str)
    new_dt2 = new_dt.join(chaos_pd, 
                        [trim(new_dt.etf_id) == chaos_pd.etf_id , trim(new_dt.etf_date) == chaos_pd.etf_date], "inner") \
    .drop(chaos_pd.etf_id).drop(chaos_pd.etf_date)
    return new_dt2
#計算上或下的值(udf)
def judge_up_down_pred(curr_price, close_price_list):
    if len(close_price_list) == 0:
        return 0.0
    else:
        prev_price = close_price_list[-1]
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred, DoubleType())
#建立預測方法
def doPredict(test_dt, etf_ids, model_dic):
    predict_res = None
    for etfid in etf_ids:
        test_data = test_dt.filter("etf_id='" + etfid + "'")
        predicts = model_dic[etfid].transform(test_data)
        if predict_res is None:
            predict_res = predicts
        else:
            predict_res = predict_res.unionAll(predicts)
    predict_res2 = predict_res.withColumn("pred_ud", judge_up_down_pred(col("prediction"), col("close_price_raw")))
    return predict_res2

In [53]:
schema = tetfp_dt2.select("etf_id", "etf_date", col("row_idx").cast("Double"), "close_price_raw", "DIF", "dif_list").schema
idx_plus = 0.0
predict_res_final = None
predict_range = [d for d in next_date_range if d not in ignore_dates]
for date in predict_range:
    next_rows = []
    idx_plus = idx_plus +1
    for row in tetf_max.collect():
        close_price_raw = row["close_price_raw"]
        close_price_raw.append(row["etf_close"])
        dif_list = row["dif_list"]
        dif = calculate_DIF_native(close_price_raw)
        dif_list.append(dif)
        next_row = (row["etf_id"], date, row["row_idx"]+idx_plus, close_price_raw, dif ,dif_list)
        next_rows.append(next_row)
    orig_dt = spark.createDataFrame(next_rows, schema)
    new_dt = create_feature(row["etf_id"], date, orig_dt, chaos_dt) #加入預測用的feature
    feature_dt_tmp = assembler.transform(new_dt)#作出 FeatureVector
    #feature_dt = scaler_model.transform(feature_dt_tmp)
    feature_dt = feature_dt_tmp.withColumn("stdFeatures", col("features")) #$$$  no scaler
    #feature_dt.select("etf_id", "etf_date", "stdFeatures").show(10, False)
    #do predict using existed model
    pred_res = doPredict(feature_dt, etf_ids, model_map)
    #取出所有預測結果作合併，以進行後續成績計算
    merged_dt = pred_res.select("etf_id", "etf_date", "prediction", "pred_ud")
    if predict_res_final is None:
        predict_res_final = merged_dt
    else:
        predict_res_final = predict_res_final.unionAll(merged_dt)
    tetf_max = pred_res.select("etf_id", "etf_date", "row_idx", col("prediction").alias("etf_close"), "close_price_raw", "dif_list")
    
predict_res_final.cache()
predict_res_final.show(10)


+------+--------+----------+-------+
|etf_id|etf_date|prediction|pred_ud|
+------+--------+----------+-------+
+------+--------+----------+-------+



In [54]:
# 評估Model的RMES
####### without scaler (depth=12, bin=64) with P_1, P_5, EMA30 MAC #########1
#(RMSE) on test data = 0.754048    accuracy = 0.626374 final score: 14.720382982416249



In [55]:
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
if DoEval:
    tetf_dt_eval = tetfp_dt2.filter("etf_date >= '" + predict_start_date + "'") \
        .select("etf_id", "etf_date", "etf_close", "close_price_raw")
    eval_dt = predict_res_final.join(tetf_dt_eval, ["etf_id", "etf_date"], "inner") \
        .withColumn("act_ud", judge_up_down_pred(col("etf_close"), col("close_price_raw")))
    eval_dt.cache()
    #eval_dt.orderBy("etf_id", "etf_date", ascending=True).show(10)

    evaluator = RegressionEvaluator(
        labelCol="etf_close", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(eval_dt)
    print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
    evaluator = MulticlassClassificationEvaluator(
        labelCol="act_ud", predictionCol="pred_ud", metricName="accuracy")
    accuracy = evaluator.evaluate(eval_dt)
    print("accuracy = %g " % accuracy)
    
    #define method to evaluate by ETF way
    final_res = eval_dt.select("etf_id", "etf_date", "prediction", "pred_ud", "etf_close", "act_ud") \
        .orderBy("etf_date", "etf_id").collect()
    weights = [0.1, 0.15, 0.2, 0.25, 0.3]
    judge_score = 0.0
    for row in final_res:
        #(up equal*0.5+((act_p - abs(pred_p - act_p))/act_p)*0.5)*weight
        act_price = row["etf_close"]
        pred_price = row["prediction"]
        eidx = next_date_range.index(row["etf_date"])
        judge_score = judge_score + \
            ((0.5 if row["pred_ud"] == row["act_ud"] else 0) + \
             ((act_price-abs(pred_price-act_price))/act_price)*0.5)*weights[eidx]
    print("final score: ", judge_score)
#numtree

IllegalArgumentException: 'requirement failed: Nothing has been added to this summarizer.'

In [ ]:
final_res = predict_res_final.orderBy("etf_id", "etf_date").collect()

In [ ]:
#export to pandas
etf_ids = []
mon_ud = []
mon_price = [] 
tue_ud = []
tue_price = []
wed_ud = []
wed_price = []
thu_ud = []
thu_price = []
fri_ud = []
fri_price = []

def encode_ud(oper_ud):
    if oper_ud == 0.0:
        return 0
    elif oper_ud == 1.0:
        return 1
    else:
        return -1

for row in final_res:
    etf_id = row["etf_id"]
    if etf_id not in etf_ids:
        etf_ids.append(etf_id)
    etf_date = row["etf_date"]
    eidx = next_date_range.index(row["etf_date"])
    if eidx == 0:
        mon_ud.append(encode_ud(row["pred_ud"]))
        mon_price.append(row["prediction"])
    elif eidx == 1:
        tue_ud.append(encode_ud(row["pred_ud"]))
        tue_price.append(row["prediction"])
    elif eidx == 2:
        wed_ud.append(encode_ud(row["pred_ud"]))
        wed_price.append(row["prediction"])
    elif eidx == 3:
        thu_ud.append(encode_ud(row["pred_ud"]))
        thu_price.append(row["prediction"])
    elif eidx == 4:
        fri_ud.append(encode_ud(row["pred_ud"]))
        fri_price.append(row["prediction"])
        
if len(mon_ud) == 0:
    mon_ud = list(0 for i in range(0,len(etf_ids)))
    mon_price = list(0.0 for i in range(0,len(etf_ids)))
if len(tue_ud) == 0:
    tue_ud = list(0 for i in range(0,len(etf_ids)))
    tue_price = list(0.0 for i in range(0,len(etf_ids)))
if len(wed_ud) == 0:
    wed_ud = list(0 for i in range(0,len(etf_ids)))
    wed_price = list(0.0 for i in range(0,len(etf_ids)))
if len(thu_ud) == 0:
    thu_ud = list(0 for i in range(0,len(etf_ids)))
    thu_price = list(0.0 for i in range(0,len(etf_ids)))
if len(fri_ud) == 0:
    fri_ud = list(0 for i in range(0,len(etf_ids)))
    fri_price = list(0.0 for i in range(0,len(etf_ids)))
    
dic = {"ETFid": etf_ids, 
       "Mon_ud": mon_ud, "Mon_cprice": mon_price,
       "Tue_ud": tue_ud, "Tue_cprice": tue_price,
       "Wed_ud": wed_ud, "Wed_cprice": wed_price,
       "Thu_ud": thu_ud, "Thu_cprice": thu_price,
       "Fri_ud": fri_ud, "Fri_cprice": fri_price
      }
final_df = pd.DataFrame(data=dic)[['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice',
                                  'Wed_ud','Wed_cprice','Thu_ud','Thu_cprice',
                                  'Fri_ud','Fri_cprice']]
final_df

In [ ]:
len(fri_price)

In [ ]:
final_df.to_csv(Path.replace("file:","") + "/etf_price_pred.csv",index=False)

In [ ]:
#ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66, 47.0, 46.96, 47.0]
print(list(0 for i in range(0,5)))
ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66]
win_len=5
print(ll[0])
print(ll[0: -1])
print(list(map(lambda x,y : x - y, ll[1:], ll[0: -1])))
for x in ll[win_len:]:
    print(x)
ema = sum(ll[:win_len])/len(ll[:win_len])
print(ema)
for price in ll[win_len:]:
    ema = (ema*(win_len-1)+price*2)/(win_len+1)
tup1, tup2 = (1,2)
print(tup1, ' ', tup2)
tup = (3,4)
print(tup[0], ' ', tup[1])
list(range(0,2))
test_dic = {}
test_dic.update({"0051": assembler})
print(test_dic['0051'])
print([x for x in next_date_range if x not in ignore_dates])